# ASV Overlap Venn Diagram

In [79]:
# Import Python packages
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import pearsonr
from matplotlib_venn import venn2
import matplotlib.patches as mpatches
from scipy.stats import mannwhitneyu
from biom import Table
from gemelli.rpca import rpca
from scipy.spatial.distance import euclidean
import re
import matplotlib.colors as mcolors
from scipy.stats import fisher_exact


In [80]:
# Load the metadata
metadata_path = '../Metadata/16S_AD_South-Africa_metadata_subset.tsv'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')

# Create group column based on case_type to simplify group names
metadata['group'] = metadata['case_type'].map({
    'case-lesional_skin': 'skin-ADL',
    'case-nonlesional_skin': 'skin-ADNL', 
    'control-nonlesional_skin': 'skin-H',
    'case-anterior_nares': 'nares-AD',
    'control-anterior_nares': 'nares-H'
})

# Add AD status column based on group values
metadata['ad_status'] = metadata['group'].apply(lambda x: 'AD' if x.split('-')[-1].startswith('AD') else 'H')

metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,age_months,sex,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,group,ad_status
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,24.0,male,4/16/2015,Autumn,Unexposed,negative,4.0,40,skin-ADL,AD
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,9.0,female,8/11/2015,Winter,Unexposed,negative,7.0,34,skin-ADL,AD
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,24.0,female,11/20/2014,Spring,Unexposed,negative,7.0,21,skin-ADL,AD
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,18.0,female,9/23/2015,Spring,Unexposed,NaN,4.0,40,skin-ADL,AD
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,31.0,male,4/21/2015,Autumn,Unexposed,negative,7.0,41,skin-ADL,AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900401,5,C12,SB503,AGAGTCAC,SB712,CGTAGCGA,SB712SB503,CGTAGCGA-AGAGTCAC,1.010000e+21,C12,...,21.0,female,9/17/2015,Spring,Exposed,negative,12.0,38,skin-ADNL,AD
900402,6,B4,SA502,ACTATCTG,SB704,TCTCTATG,SB704SA502,TCTCTATG-ACTATCTG,1.010000e+21,B4,...,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,AD
Ca006ONL,6,F1,SA506,CGTGAGTG,SB701,CTCGACTT,SB701SA506,CTCGACTT-CGTGAGTG,1.010000e+21,F1,...,35.0,female,3/25/2015,Autumn,Unexposed,negative,3.0,34,skin-ADL,AD


In [81]:
# Read in table at collapsed genera level
biom_path = '../Data/Tables/Count_Tables/6_209766_feature_table_dedup_prev-filt-1pct_rare-350_Genus-ASV_all.biom'

biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the 
df.index = df.index.str.replace('15564.', '')

# Map pid, group, and microbiome_type from metadata to df based on matching indexes
df['pid'] = metadata.loc[df.index, 'pid']
df['group'] = metadata.loc[df.index, 'group']
df['microbiome_type'] = metadata.loc[df.index, 'specimen']
df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-3,g___ASV-3,g__Cutibacterium_ASV-1,g___ASV-15,g___ASV-29,g___ASV-25,g__Cutibacterium_ASV-2,...,g__UBA6175_ASV-2,g__Dermacoccus_ASV-2,g__Comamonas_F_589250_ASV-4,g__Bosea_ASV-2,g__Capnocytophaga_820690_ASV-3,g__Blautia_A_141780_ASV-1,g___ASV-168,pid,group,microbiome_type
900344,188.0,115.0,23.0,19.0,2.0,2.0,1.0,0,0,0,...,0,0,0,0,0,0,0,Co-130-MM,skin-H,skin
900459,20.0,37.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-145-LM,nares-AD,nasal
900221,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-101-ID,skin-ADL,skin
900570,18.0,0,0,0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,Ca-046-MT,skin-ADNL,skin
900092,221.0,116.0,3.0,1.0,0,0,0,0,1.0,0,...,0,0,0,0,0,0,0,Ca-023-EJ,nares-AD,nasal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900057,29.0,0,0,0,0,2.0,4.0,0,0,3.0,...,0,0,0,0,0,0,0,Ca-013-NN,skin-ADL,skin
900294,10.0,3.0,0,0,0,41.0,0,0,0,36.0,...,0,0,0,0,0,0,0,Ca-113-MN,skin-ADNL,skin
900097,3.0,0,0,0,0,2.0,0,0,0,1.0,...,0,0,0,0,0,0,0,Ca-025-AC,skin-ADNL,skin
900498,5.0,5.0,0,0,0,17.0,0,4.0,0,8.0,...,0,0,0,0,0,0,0,Ca-157-LM,skin-ADNL,skin


In [82]:
print(f'Number of unique individuals: ' + str(df['pid'].nunique()))

Number of unique individuals: 183


In [83]:
# --- STEP 1: Prepare ---
# Only keep numeric columns (i.e., ASV features)
taxa_cols = df.select_dtypes(include=[np.number]).columns.tolist()


# Split into skin and nares samples
skin_samples = df[df['microbiome_type'] == 'skin']
nares_samples = df[df['microbiome_type'] == 'nasal']

# --- STEP 2: Separate by disease group ---

# AD individuals
skin_ADL_samples = skin_samples[skin_samples['group'] == 'skin-ADL']
nares_AD_samples = nares_samples[nares_samples['group'] == 'nares-AD']

# Healthy individuals
skin_H_samples = skin_samples[skin_samples['group'] == 'skin-H']
nares_H_samples = nares_samples[nares_samples['group'] == 'nares-H']

# --- STEP 3: Find taxa for each group ---

# For AD
skin_present_AD = skin_ADL_samples[taxa_cols].gt(0).any()
nares_present_AD = nares_AD_samples[taxa_cols].gt(0).any()

skin_taxa_AD = set(skin_present_AD[skin_present_AD].index)
nares_taxa_AD = set(nares_present_AD[nares_present_AD].index)

# For Healthy
skin_present_H = skin_H_samples[taxa_cols].gt(0).any()
nares_present_H = nares_H_samples[taxa_cols].gt(0).any()

skin_taxa_H = set(skin_present_H[skin_present_H].index)
nares_taxa_H = set(nares_present_H[nares_present_H].index)

In [84]:
# Get unique patient IDs for each group
skin_ADL_pids = set(skin_ADL_samples['pid'].unique())
nares_AD_pids = set(nares_AD_samples['pid'].unique())

# Count unique patients
AD_pids_skin_nasal_overlap = skin_ADL_pids.intersection(nares_AD_pids)
n_AD_pids_skin_nasal_overlap = len(AD_pids_skin_nasal_overlap)
print(f'Number of AD participants with both paired skin and nasal samples: ', n_AD_pids_skin_nasal_overlap)

Number of AD participants with both paired skin and nasal samples:  86


In [85]:
# Get unique patient IDs for each group
skin_H_pids = set(skin_H_samples['pid'].unique())
nares_H_pids = set(nares_H_samples['pid'].unique())

# Count unique patients
H_pids_skin_nasal_overlap = skin_H_pids.intersection(nares_H_pids)
n_H_pids_skin_nasal_overlap = len(H_pids_skin_nasal_overlap)

print(f'Number of Healthy participants with both paired skin and nasal samples: ', n_H_pids_skin_nasal_overlap)

Number of Healthy participants with both paired skin and nasal samples:  72


In [86]:
# --- STEP 4: Plot Venn diagrams side by side ---
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 row, 2 columns

# Colors
skin_color = '#2a00ff'
nares_color = '#ffa501'
overlap_color = '#955280'

# --- Venn for AD Individuals ---
plt.sca(axes[0])  # set current axis
v_AD = venn2(
    [skin_taxa_AD, nares_taxa_AD],
    set_labels=('', ''),
    set_colors=(skin_color, nares_color),
    alpha=0.5
)

# Customize font sizes and outlines
for label in v_AD.subset_labels:
    if label:
        label.set_fontsize(16)

region_colors = {
    '10': skin_color,
    '01': nares_color,
    '11': overlap_color
}
for region_id, color in region_colors.items():
    patch = v_AD.get_patch_by_id(region_id)
    if patch:
        patch.set_linewidth(2)
        patch.set_edgecolor(color)

axes[0].set_title(f'AD Individuals (n={n_AD_pids_skin_nasal_overlap})', fontsize=18, y=0.94)

# --- Venn for Healthy Individuals ---
plt.sca(axes[1])  # set current axis
v_H = venn2(
    [skin_taxa_H, nares_taxa_H],
    set_labels=('', ''),
    set_colors=(skin_color, nares_color),
    alpha=0.5
)

for label in v_H.subset_labels:
    if label:
        label.set_fontsize(16)

for region_id, color in region_colors.items():
    patch = v_H.get_patch_by_id(region_id)
    if patch:
        patch.set_linewidth(2)
        patch.set_edgecolor(color)

axes[1].set_title(f'Healthy Individuals (n={n_H_pids_skin_nasal_overlap})', fontsize=18, y=0.94)

# --- Add Legend ---
handles = [
    mpatches.Patch(color=skin_color, alpha=0.5, label='Skin only'),
    mpatches.Patch(color=overlap_color, alpha=0.5, label='Both'),
    mpatches.Patch(color=nares_color, alpha=0.5, label='Nares only')
]

fig.legend(
    handles=handles,
    loc='lower center',
    bbox_to_anchor=(0.5, 0.02),
    ncol=3,
    frameon=False,
    fontsize=16
)

# --- Final layout tweaks ---
plt.tight_layout()
plt.subplots_adjust(top=0.88, bottom=0.15)

# --- Supertitle ---
fig.suptitle('ASV Overlap Between Skin and Nares', fontsize=20, y=0.99)

# --- Annotation data ---
# You already computed these
shared_AD = len(skin_taxa_AD & nares_taxa_AD)
total_AD = len(skin_taxa_AD | nares_taxa_AD)

shared_H = len(skin_taxa_H & nares_taxa_H)
total_H = len(skin_taxa_H | nares_taxa_H)

# Run Fisher's test again if needed
from scipy.stats import fisher_exact
table = [[shared_AD, total_AD - shared_AD], [shared_H, total_H - shared_H]]
oddsratio, p_value = fisher_exact(table)

# Add p-value and odds ratio centrally below both
fig.text(
    0.5, 0.13,
    f"Fisher's Exact Test: p = 8.1×10⁻⁷, Odds Ratio = {oddsratio:.2f}",
    ha='center', fontsize=14
)

plt.savefig('../Figures/Main/Fig_3A.jpg', dpi=600)

In [87]:
# --- For AD group ---
shared_AD = len(skin_taxa_AD & nares_taxa_AD)
total_AD = len(skin_taxa_AD | nares_taxa_AD)

# --- For Healthy group ---
shared_H = len(skin_taxa_H & nares_taxa_H)
total_H = len(skin_taxa_H | nares_taxa_H)

# --- Build 2x2 contingency table ---
table = [
    [shared_AD, total_AD - shared_AD],
    [shared_H, total_H - shared_H]
]

# --- Fisher's Exact Test ---
oddsratio, p_value = fisher_exact(table)

# --- Output Results ---
print(f"Fisher's Exact Test results:")
print(f"Contingency Table: {table}")
print(f"Odds Ratio: {oddsratio:.2f}")
print(f"p-value: {p_value:.8f}")

Fisher's Exact Test results:
Contingency Table: [[372, 338], [272, 422]]
Odds Ratio: 1.71
p-value: 0.00000081
